# PseudoWriter: A Pythonic Journey into Markov Chain Text Generation

Welcome to PseudoWriter, a project that explores the fascinating world of text generation using simple yet powerful algorithms. Building upon the concepts from our previous Java implementation, this Jupyter notebook delves into creating a Markov Chain-based text generator in Python.

Our goal is to generate new text that mimics the style and vocabulary of a given source. For this Python version, we'll be training our model on the monumental work of Victor Hugo: **_Les Misérables_**! 🇫🇷📚

This notebook will guide you through the process, from data loading and model learning to text generation, with clear explanations and type-hinted code.

In [1]:
import os
from numpy import random

# Define special tokens for the Markov Chain
START: str = "<START>" # Marks the beginning of a sequence
PAR: str = "<PAR>"     # Marks a paragraph break
END: str = "<END>"       # Marks the end of a sequence

# Configuration for the Markov Chain model
mem_depth: int = 4 # The length of the prefix (number of words to look back)
novel: str = "miserables" # The directory containing the source text files

# Discover and load the source text files
nb_toms: int = len(os.listdir(novel)) # Number of 'tomes' (volumes) in the novel
toms_paths: list[str] = [os.path.join(novel, f"t{i}.txt") for i in range(1, nb_toms + 1)]

print(f"Discovered tomes paths: {toms_paths}")

toms: list[str] = []
for tom_path in toms_paths:
    with open(tom_path, 'r', encoding='utf-8') as file:
        toms.append(file.read())

print(f"Loaded tomes (first 30 chars): {[tom[:30] + '...' for tom in toms]}")

Discovered tomes paths: ['miserables\\t1.txt', 'miserables\\t2.txt', 'miserables\\t3.txt', 'miserables\\t4.txt', 'miserables\\t5.txt']
Loaded tomes (first 30 chars): ['En 1815, M. Charles-François-B...', 'L’an dernier (1861), par une b...', 'Paris a un enfant et la forêt ...', '1831 et 1832, les deux années ...', 'Les deux plus mémorables barri...']


## Building the Markov Model: The Learning Phase 🧠

The core of our text generator lies in building a statistical model from the input text. This model, often represented as a dictionary or hash map, stores the probabilities of words appearing after specific sequences of preceding words (our prefixes).

We define a `learn` function that iterates through all the provided text volumes and populates this `memory` dictionary. Each key in `memory` is a `tuple` representing a prefix, and its value is a `list` of words that have been observed to follow that prefix in the text.

The `learn_tom` function processes a single volume, updating the `memory` dictionary. It uses a sliding window approach to extract prefixes and their subsequent words. Special `START` and `END` tokens are used to mark the beginning and end of the text sequences, allowing the generator to start and stop gracefully. The `PAR` token is used to denote paragraph breaks, preserving the structure of the original text.

In [2]:
def learn(toms: list[str], mem_depth: int) -> dict[tuple[str, ...], list[str]]:
    """
    Learns the Markov chain model from a list of text volumes.

    Args:
        toms: A list of strings, where each string is a volume of the source text.
        mem_depth: The depth of the Markov chain (length of the prefix).

    Returns:
        A dictionary representing the Markov chain, mapping prefixes to lists of following words.
    """
    memory: dict[tuple[str, ...], list[str]] = {}

    for tom in toms:
        learn_tom(tom, memory, mem_depth)

    return memory

def learn_tom(tom: str, memory: dict[tuple[str, ...], list[str]], mem_depth: int) -> None:
    """
    Learns the Markov chain model from a single text volume.

    Args:
        tom: The text content of a single volume.
        memory: The dictionary representing the Markov chain (modified in-place).
        mem_depth: The depth of the Markov chain (length of the prefix).
    """
    # Initialize the prefix with START tokens
    prefix: tuple[str, ...] = (START,) * mem_depth

    # Helper function to update the memory dictionary
    def prefix_update(word: str) -> None:
        if prefix not in memory:
            memory[prefix] = [word]
        else:
            memory[prefix].append(word)

    # Split the text into words and iterate
    for word in tom.split(" "):
        prefix_update(word)
        # Shift the prefix: remove the first word, add the new word at the end
        prefix = prefix[1:] + (word,)

    # Mark the end of the text sequence
    prefix_update(END)

def mem_preview(memory: dict[tuple[str, ...], list[str]], n: int) -> None:
    """
    Prints a preview of the learned Markov chain memory.

    Args:
        memory: The Markov chain dictionary.
        n: The number of entries to preview.
    """
    print("\nMemory Preview:")
    for i, (prefix, words) in enumerate(memory.items()):
        if i >= n:
            break
        print(f"\t{prefix}: {words}")
    print("\t...")

# Learn the model from the loaded tomes
memory: dict[tuple[str, ...], list[str]] = learn(toms, mem_depth)

# Display a preview of the learned memory
mem_preview(memory, 30)


Memory Preview:
	('<START>', '<START>', '<START>', '<START>'): ['En', 'L’an', 'Paris', '1831', 'Les']
	('<START>', '<START>', '<START>', 'En'): ['1815,']
	('<START>', '<START>', 'En', '1815,'): ['M.']
	('<START>', 'En', '1815,', 'M.'): ['Charles-François-Bienvenu']
	('En', '1815,', 'M.', 'Charles-François-Bienvenu'): ['Myriel']
	('1815,', 'M.', 'Charles-François-Bienvenu', 'Myriel'): ['était']
	('M.', 'Charles-François-Bienvenu', 'Myriel', 'était'): ['évêque']
	('Charles-François-Bienvenu', 'Myriel', 'était', 'évêque'): ['de']
	('Myriel', 'était', 'évêque', 'de'): ['Digne.']
	('était', 'évêque', 'de', 'Digne.'): ['C’était']
	('évêque', 'de', 'Digne.', 'C’était'): ['un']
	('de', 'Digne.', 'C’était', 'un'): ['vieillard']
	('Digne.', 'C’était', 'un', 'vieillard'): ['d’environ']
	('C’était', 'un', 'vieillard', 'd’environ'): ['soixante-quinze']
	('un', 'vieillard', 'd’environ', 'soixante-quinze'): ['ans\xa0;']
	('vieillard', 'd’environ', 'soixante-quinze', 'ans\xa0;'): ['il']
	('d’environ'

## Generating New Text: The Creative Phase ✍️

With our Markov model (the `memory` dictionary) built, we can now generate new text that emulates the style of _Les Misérables_. The `gen` function orchestrates this creative process.

The generation starts with an initial prefix composed entirely of `START` tokens. Then, in a loop, it repeatedly:

1.  Looks up the current prefix in the `memory` to find all possible next words.
2.  Randomly selects one word from this list of possibilities.
3.  Appends the chosen word to our `gen_tom` (generated text) string.
4.  Updates the current prefix by shifting out the oldest word and adding the newly chosen word.

This process continues until the `END` token is randomly selected, signaling the completion of the generated text. The `PAR` token is handled specifically to insert newlines, preserving paragraph structure.

In [5]:
def gen(memory: dict[tuple[str, ...], list[str]], mem_depth: int) -> str:
    """
    Generates new text based on the learned Markov chain model.

    Args:
        memory: The Markov chain dictionary.
        mem_depth: The depth of the Markov chain (length of the prefix).

    Returns:
        The newly generated text as a string.
    """
    # Initialize the prefix with START tokens
    prefix: tuple[str, ...] = (START,) * mem_depth
    gen_tom: str = ""

    while True:
        # Get the list of possible next words for the current prefix
        possible_next_words: list[str] = memory.get(prefix, [END]) # Default to END if prefix not found
        
        # Randomly choose a word from the possibilities
        rand_word: str = random.choice(possible_next_words)
        
        # If the END token is chosen, stop generation
        if rand_word == END:
            break
        
        gen_tom += rand_word + " " # Append the word and a space
        
        # Handle paragraph breaks
        # if rand_word == PAR:
        #     gen_tom += f"\n{PAR}\n" # Add two newlines for a paragraph break
        # else:
        #     gen_tom += rand_word + " " # Append the word and a space
        
        # Update the prefix for the next iteration
        prefix = prefix[1:] + (rand_word,)
    
    return gen_tom.strip() # Remove any trailing space

# Generate a new 'tome'
new_tom: str = gen(memory, mem_depth)

# Save the generated text to a new file
new_tom_filename: str = os.path.join(novel, f"t{nb_toms + 1}.txt")
with open(new_tom_filename, "w", encoding='utf-8') as f:
    f.write(new_tom)

print("\n--- Generated Tome ---\n")
print(new_tom)
print(f"\n--- Saved to: {new_tom_filename} ---")


--- Generated Tome ---

Paris a un enfant et la forêt a un oiseau ; l’oiseau s’appelle le moineau ; l’enfant s’appelle le gamin. <PAR> Accouplez ces deux idées qui contiennent, l’une toute la fournaise, l’autre toute l’aurore, choquez ces étincelles, Paris, l’enfance ; il en jaillit un petit être. Homuncio, dirait Plaute. <PAR> Ce petit être est joyeux. Il ne mange pas tous les jours et de faire courir les personnes comme cela ! <PAR> Marius s’était rendu au Luxembourg. <PAR> La jeune fille avait touché son semestre. <PAR> Et puis ce paquet d’habits préparés d’avance pour la petite, tout cela était singulier ; il y avait bien des mystères là-dessous. On ne lâche pas des mystères quand on les tient. Les secrets des riches sont des éponges pleines d’or ; il faut savoir les presser. Toutes ces pensées lui tourbillonnaient dans le cerveau. <PAR> — Je suis un cloporte, monseigneur. <PAR> Chaque maison de ce genre a ses particularités. Au commencement de ce siècle, Écouen était un de ces ho

## Conclusion and Next Steps 🎉

This notebook demonstrates how a relatively simple Markov Chain model can generate text that, especially with a sufficient `mem_depth`, can surprisingly capture the stylistic nuances of a large source text like _Les Misérables_. While it doesn't truly "understand" language, it effectively mimics patterns, creating a fascinating illusion of coherence.

### How to Run This Notebook

1.  **Clone the Repository:** If you haven't already, clone the `pseudoWriter` repository from GitHub.
    ```bash
    git clone https://github.com/KpihX/pseudoWriter.git
    cd pseudoWriter/pseudo_writer
    ```
2.  **Ensure Dependencies:** Make sure you have `numpy` installed (`pip install numpy`).
3.  **Open with Jupyter:** Launch Jupyter Notebook or JupyterLab in this directory and open `Pseudo_Writer.ipynb`.
4.  **Run Cells:** Execute the cells sequentially. The generated text will be printed to the output and saved as a new `.txt` file in the `miserables/` directory.

### Experiment! 🧪

Feel free to modify `mem_depth` to observe its impact on the generated text's coherence. You can also replace the `miserables` directory with your own text files (ensure they are `.txt` and properly formatted with `PAR` markers for paragraph breaks if desired) to train the model on different sources.

### Project Repository

Explore the full project, including the Java implementation, on GitHub:

[https://www.github.com/KpihX/pseudoWriter](https://www.github.com/KpihX/pseudoWriter)

Happy generating! 🤖✍️